In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import logging
import glob
import re
import click
import numpy as np
import pandas as pd
import xarray as xr
import suxarray as sx
import suxarray.helper
import hvplot.xarray

In [ ]:
case_name = "2016_base"

In [ ]:
chunks = {"time": 2}

In [ ]:
path_study = Path(f"/mnt/local/simulations/") / case_name

In [ ]:
date_base = pd.to_datetime("2016-04-27")
year = date_base.year
date_start = pd.to_datetime(f"{year}-07-01")
date_end = pd.to_datetime(f"{year}-11-25")

In [ ]:
day_start = (date_start - date_base).days + 1
day_end = (date_end - date_base).days + 1

In [ ]:
output_items = {
    "out2d": "outputs/out2d_{}.nc",
}
chunks = {"time": 2}
list_nc_paths = {
    k: [v.format(i) for i in range(day_start, day_end + 1)]
    for k, v in output_items.items()
}

list_ds = []
for _, v in list_nc_paths.items():
    list_paths = [str(path_study / v[i]) for i in range(len(v))]
    ds = suxarray.helper.read_schism_nc(list_paths, chunks=chunks)
    list_ds.append(ds)

ds = xr.merge(list_ds)
if sx.get_topology_variable(ds) is None:
    ds = sx.add_topology_variable(ds)
ds = sx.coerce_mesh_name(ds)

sx_ds = sx.Dataset(ds, sxgrid=sx.Grid(ds))

In [ ]:
sx_ds

In [ ]:
path_postprocess = path_study / "postprocess"

In [ ]:
chunks = {"time": 48}
path_depth_averaged_salinity_at_face = path_postprocess / "depth_averaged_salinity_at_face.nc"
ds_depth_averaged_salinity_at_face = xr.open_mfdataset(path_depth_averaged_salinity_at_face, chunks=chunks)
da_depth_averaged_salinity_at_face = ds_depth_averaged_salinity_at_face["depth_averaged_salinity_at_face"]

In [ ]:
da_depth_averaged_salinity_at_face

In [ ]:
def shift_30min_up(da):
    """Shift time coordinate 30 minutes up"""
    return da.assign_coords(time=da.coords["time"] - pd.to_timedelta("30m"))


In [ ]:
da_salinity_less_than_6 = xr.where(da_depth_averaged_salinity_at_face < 6.0, 1.0, 0.0)
da_salinity_less_than_6_daily = da_salinity_less_than_6.resample(time="1D", origin="start").mean()

In [ ]:
da_salinity_less_than_7 = xr.where(da_depth_averaged_salinity_at_face < 7.0, 1.0, 0.0)
da_salinity_less_than_7_daily = da_salinity_less_than_7.resample(time="1D", origin="start").mean()

In [ ]:
da_face_areas = sx_ds.sxgrid.compute_face_areas()

In [ ]:
da_salinity_less_than_6_daily.dot(da_face_areas)

## Subregion